# Projekt PRIAD

## Rozpoznawanie autora utworu literackiego na podstawie treści utworu z wykorzystaniem worka słów.

### Stanisław Maliński, Jan Stachurski

Importy

In [35]:
import os
import pandas as pd
import numpy as np

Wczytywanie danych

In [3]:
data_path = "./data/"
file_list = os.listdir(data_path)
func = lambda  a : a.replace(".txt", "").split("-")
count_size = lambda a : os.stat(a).st_size
data = [[func(title)[0], func(title)[1], data_path + title, count_size(data_path + title)] for title in file_list]
df = pd.DataFrame(data, columns=["Author", "Name of The work", "FilePath", "FileSize"])

Przedstawiam zestaw uczący. Mamy tutaj pozycję kilku Polskich autorów.

In [4]:
print(df.drop(columns=["Name of The work", "FilePath"]).groupby("Author").sum())

                   FileSize
Author                     
AdamMickiewicz       516084
HenrykSienkiewicz    867390


Wczytanie stop-słów:

In [7]:
delimeters = ["\\n", "'"]
stop_words = []
with open("./stopyPL.txt", encoding="utf8") as f:
    for line in f:
        word = repr(line)
        for c in delimeters:
            word = word.replace(c, "")
        stop_words.append(word)

Projekt ten używa worka słów, który to przechowuje informacje na temat częstości występowania słów w danym dokumencie. Definiujemy tu dwie funckje jedna do standaryzacji słów tzn. usunięcia ewentualnych znaków graficznych takich jak kropka czy przecinek oraz ustawienie wielkości liter na małe aby słowa "Który" oraz "który" były zliczane jako to same słowo.
Druga funkcja służy do budowania worka słów. Zlicza najpierw pojedyńcze przypadki wystąpień, a następnie zlicza częstość wystąpień danego słowa.

In [21]:
def normalize_word(word):
    word = word.lower()
    special_char = [".", ",", "-", "?", "(", ")", "!", "\\", "\"", ":", ";", "*"]
    for char in special_char:
        word = word.replace(char, "")
    return word

def remove_stop_words(word):
    pass

def generate_word_bag(FileName, rmstops=True):
    word_bag = {}
    over_all_words = 0
    with open(FileName, "r", encoding="utf8") as f:
        for line in f:
            words = line.split()
            words = [normalize_word(word) for word in words]
            for word in words:
                if word != '':
                    over_all_words += 1
                    if word in word_bag.keys():
                        word_bag[word]['Count'] += 1
                    else:
                        word_bag[word] = {'Count':1, 'Frequent':1, 'Is stop word':False}
                                    
        f.close()
    for word in word_bag.keys():
        word_bag[word]['Frequent'] = word_bag[word]['Count'] / over_all_words
    return word_bag



In [24]:
file = "./data/AdamMickiewicz-Burza.txt"
dictionary = generate_word_bag(file)
slowa = dictionary.keys()
slowo = list(slowa)[2]

print("W pliku: \"{}\" Słowo: \"{}\" występowało z częstością: {:.2e}".format(file, slowo, dictionary[slowo]['Frequent']))

W pliku: "./data/AdamMickiewicz-Burza.txt" Słowo: "burza" występowało z częstością: 9.62e-03


In [29]:
def head(dictionary, start=0, end=None, step=1, fname=None):
    words = list(dictionary.keys())
    if end is None:
        end = min(5, len(words))
    if fname is not None:
        print("W pliku: \"{}\"".format(fname))
    for word in words[start:end:step]:
        print("Słowo: \"{}\" występowało z częstością: {:.2e}".format(word,dictionary[word]['Frequent']), sep="")

In [30]:
file = df["FilePath"][1]
dictionary = generate_word_bag(file, rmstops=False)
head(dictionary, end=30, fname="data/AdamMickiewicz-Dziady.txt")

W pliku: "data/AdamMickiewicz-Dziady.txt"
Słowo: "dziady" występowało z częstością: 2.44e-03
Słowo: "poema" występowało z częstością: 6.09e-04
Słowo: "część" występowało z częstością: 3.05e-04
Słowo: "ii" występowało z częstością: 3.05e-04
Słowo: "jest" występowało z częstością: 2.13e-03
Słowo: "to" występowało z częstością: 1.37e-02
Słowo: "nazwisko" występowało z częstością: 3.05e-04
Słowo: "uroczystości" występowało z częstością: 6.09e-04
Słowo: "obchodzonej" występowało z częstością: 3.05e-04
Słowo: "dotąd" występowało z częstością: 1.22e-03
Słowo: "między" występowało z częstością: 3.05e-04
Słowo: "pospólstwem" występowało z częstością: 3.05e-04
Słowo: "w" występowało z częstością: 2.50e-02
Słowo: "wielu" występowało z częstością: 3.05e-04
Słowo: "powiatach" występowało z częstością: 3.05e-04
Słowo: "litwy" występowało z częstością: 3.05e-04
Słowo: "prus" występowało z częstością: 3.05e-04
Słowo: "i" występowało z częstością: 2.62e-02
Słowo: "kurlandii" występowało z częstością: 3

In [32]:
def show_most_frequent(dic ,threshold=0):
    po_czestosci = dict(sorted(dic.items(), key=lambda x: int(x[1]['Count']), reverse = True))
    for index, word in enumerate(po_czestosci.keys()):
        if threshold == 0 and index < 10:
            print(word + " -> " + str(po_czestosci[word]['Count']))
        elif threshold == 0:
            break
        elif po_czestosci[word]['Count'] > threshold:
            print(word + " -> " + str(po_czestosci[word]['Count']))
        else:
            break
show_most_frequent(dictionary)

nie -> 98
i -> 86
w -> 82
z -> 71
a -> 48
to -> 45
się -> 45
na -> 42
guślarz -> 38
chór -> 36


In [33]:
def merge_dics_to_df(dics, labels=None):
    tab = []
    word_found = []
    for dic in dics:
        for word in dic.keys():
            if not word in word_found:
                word_found.append(word)
                rec = []
                for dictionary in dics:
                    if word in dictionary.keys():
                        rec.append(dictionary[word])
                    else:
                        rec.append(0)
                tab.append(rec)
    
    if labels is not None: 
        df = pd.DataFrame(tab, columns=labels, index=word_found)
    else:
        df = pd.DataFrame(tab)
        
    return df
    
df = merge_dics_to_df([{'1': 1, '2': 2}, 
                        {'3': 3, '2': 2}, 
                        {'3': 3, '2': 2, '4': 4, '7':7}], 
                       ["Pierwszy słownik", "Drugi słownik", "Trzeci słownik"])
print(df)

   Pierwszy słownik  Drugi słownik  Trzeci słownik
1                 1              0               0
2                 2              2               2
3                 0              3               3
4                 0              0               4
7                 0              0               7


In [36]:
def tf_idf(data):   
    ile_slow = data.count(0)[0]
    tf = np.empty((ile_slow,len(data.columns)))
    for i in range(0,len(data.columns)):
        tf[:,i] = np.array(data.iloc[:,i]/data.sum()[i])
                           
    #print((data>0).head())

    #print(np.array((data>0)*1).sum(axis=1))
    
    idf = np.log(len(data.columns)/np.array((data>0)*1).sum(axis=1))

    data['w ilu'] = (np.array((data>0)*1).sum(axis=1)).T
    data['idf'] = idf.T
    
tf_idf(df)
print(df)

   Pierwszy słownik  Drugi słownik  Trzeci słownik  w ilu       idf
1                 1              0               0      1  1.098612
2                 2              2               2      3  0.000000
3                 0              3               3      2  0.405465
4                 0              0               4      1  1.098612
7                 0              0               7      1  1.098612


In [37]:
def drop_if(data, idf_threshold):
    data = data[data.idf <= idf_threshold]
    return data

print(drop_if(df, 1))

   Pierwszy słownik  Drugi słownik  Trzeci słownik  w ilu       idf
2                 2              2               2      3  0.000000
3                 0              3               3      2  0.405465


In [38]:
def drop_if(data, idf_threshold):
    data = data[data.idf <= idf_threshold]
    return data

print(drop_if(df, 1))

   Pierwszy słownik  Drugi słownik  Trzeci słownik  w ilu       idf
2                 2              2               2      3  0.000000
3                 0              3               3      2  0.405465


Teraz musimy stworzyć model, który będzie klasyfikował do jakiego autora należy dany utwór.